Task 1:
Complete the following steps to establish a connection pool:

To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.

To find the information on the error, import the Error class from MySQL Connector/Python.

Define your database configurations as a Python dictionary object called dbconfig.

Establish a connection pool [pool_name = pool_b] with two connections. 

Implement error handling using a try-except block in case the connection fails.

In [1]:
"""
To create a connection pool, import MySQLConnectionPool class from MySQL Connector/Python.
To find the information on the error, import the Error class from MySQL Connector/Python.
"""

from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import mysql.connector as connector
import pandas as pd

In [2]:
"""
Define your database configurations as a Python dictionary object called dbconfig.
"""

dbconfig = {
    'user' : 'root',
    'password' : '******',
    'database' : 'meta_pytomysql_test'
}

In [3]:
"""
Establish a connection pool [pool_name = pool_b] with two connections. 
Implement error handling using a try-except block in case the connection fails.
"""

try:
    pool = MySQLConnectionPool(
        pool_name = 'pool_b',
        pool_size =  2,
        **dbconfig
    )
    print('The connection pool is created with a name: ', pool.pool_name)
    print('The pool size is: ', pool.pool_size)

except Error as err:
    print('Error code: ', err.errno)
    print('Error message: ', err.msg)

The connection pool is created with a name:  pool_b
The pool size is:  2


Task 2:
Three guests are trying to book dinner slots simultaneously. Get the connections from pool_b and insert the following data in the Bookings table:

TIP: You need to add a connection to connect the third guest.

Guest 1:
Table Number: 8
First Name: Anees
Last Name: Java
Booking Time: 18:00
EmployeeID: 6

Guest 2:
Table Number: 5
First Name: Bald
Last Name: Vin
Booking Time: 19:00
EmployeeID: 6

Guest 3:
Table Number: 12
First Name: Jay
Last Name: Kon
Booking Time: 19:30 
EmployeeID: 6

Return all the connections back to the pool. 

TIP: The pool size is two. However, you have three connected users. You can only return two connections. Returning a third connection will raise a PoolError. Use try-except to print the error message.  

In [4]:
guests = ['guest1', 'guest2', 'guest3']

In [5]:
for guest in guests:

    try:
        connection = pool.get_connection()
        print(f'The user {guest} is connected using the id {connection.connection_id}')
    except:
        print(f"It's not possible to establish a connection because the pool {pool.pool_name} has used {pool.pool_size} connections.")
        print(f"Trying to make a connection manually...")

        connection = connector.connect(**dbconfig)
        print('A connection with the database was established.')
        pool.add_connection(cnx=connection)

        print(f"{guest} was edded in the pool using the id {connection.connection_id}")

The user guest1 is connected using the id 93
The user guest2 is connected using the id 94
It's not possible to establish a connection because the pool pool_b has used 2 connections.
Trying to make a connection manually...
A connection with the database was established.
guest3 was edded in the pool using the id 95


Task 3:
Create a report containing the following information:

The name and EmployeeID of the Little Lemon manager.

The name and role of the employee who receives the highest salary.

The number of guests booked between 18:00 and 20:00.

The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

In [6]:
# Task 3:
try:
    cursor1 = connection.cursor()
    print('cursor created.')
except:
    print("Can't create the cursor")

cursor created.


In [7]:
# Task 3.1: The name and EmployeeID of the Little Lemon manager.

query_report1 = """SELECT name AS 'Name', EmployeeID FROM employees WHERE role = 'Manager';"""
cursor1.execute(query_report1)

results1 = cursor1.fetchall()
columns1 = cursor1.column_names

df1 = pd.DataFrame(data=results1, columns=columns1)
df1.style.hide(axis='index')

Name,EmployeeID
Mario Gollini,1


In [8]:
# Task 3.2: The name and role of the employee who receives the highest salary.

query_report2 = """SELECT name AS 'Name', role AS 'Role' FROM employees WHERE annual_salary = (SELECT MAX(annual_salary) FROM employees);"""
cursor1.execute(query_report2)

results2 = cursor1.fetchall()
columns2 = cursor1.column_names

df2 = pd.DataFrame(data=results2, columns=columns2)
df2.style.hide(axis='index')

Name,Role
Mario Gollini,Manager


In [9]:
# Task 3.3: The number of guests booked between 18:00 and 20:00.

query_report3 = """SELECT COUNT(*) AS 'Bookings: 18:00 - 20:00' FROM bookings WHERE bookingSlot BETWEEN '18:00' AND '20:00';"""
cursor1.execute(query_report3)

results3 = cursor1.fetchall()
columns3 = cursor1.column_names

df3 = pd.DataFrame(data=results3, columns=columns3)
df3.style.hide(axis='index')

Bookings: 18:00 - 20:00
4


In [10]:
# Task 3.4: The full name and BookingID of all guests waiting to be seated with the receptionist in sorted order with respect to their BookingSlot.

query_report4 = """SELECT CONCAT(guestFirstName, ' ',guestLastName) name,bookingSlot ,bookingID FROM bookings ORDER BY bookingSlot;"""
cursor1.execute(query_report4)

results4 = cursor1.fetchall()
columns4 = cursor1.column_names

df4 = pd.DataFrame(data=results4, columns=columns4)
df4.style.hide(axis='index')

name,bookingSlot,bookingID
Vanessa McCarthy,0 days 15:00:00,3
Marcos Romero,0 days 17:30:00,4
Hiroki Yamane,0 days 18:30:00,5
Anna Iversen,0 days 19:00:00,1
Joakim Iversen,0 days 19:00:00,2
Diana Pinto,0 days 20:00:00,6


Task 4:
Create a stored procedure named BasicSalesReport that returns the following statistics: 

Total sales
Average sale
Minimum bill paid
Maximum bill paid

In [11]:
# Task 4: CREATE THE PROCEDURE
cursor1.execute("""DROP PROCEDURE IF EXISTS BasicSalesReport;""")

procedure_BasicSalesReport = """
CREATE PROCEDURE BasicSalesReport()
BEGIN

SELECT SUM(billAmount) 'Total sales',
AVG(billAmount) 'Average sale',
MIN(billAmount) 'Minimum bill paid',
MAX(billAmount) 'Maximum bill paid' 
FROM orders;

END
"""
cursor1.execute(procedure_BasicSalesReport)
print('Procedure created.')

Procedure created.


In [12]:
# Task 4: Call the procedure BasicSalesReport.

cursor1.callproc('BasicSalesReport')
results5 = next(cursor1.stored_results())
dataset5 = results5.fetchall()
columns5 = results5.column_names

df5 = pd.DataFrame(data=dataset5, columns=columns5)
df5.style.hide(axis='index')

Total sales,Average sale,Minimum bill paid,Maximum bill paid
243,48.6000,37,86


Task 5:
Little Lemon needs to display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next. To complete this task, carry out the following steps:

Get a connection from the pool.

Create a buffered cursor.

Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 

Returned the connection back to the pool.

The output should be as follows:

[BookingSlot]

[Guest_name]

[Assigned to: Employee Name [Employee Role]]

In [13]:
# Task 5.1: Get a connection from the pool.
try:
    connection2 = pool.get_connection()
    print('Connected...')
except:
    print('Not possible to create a connection')

Connected...


In [18]:
# confirm the connection with the pool:

print(f"Connection established with the pool {connection2.pool_name}, with ID: {connection2.connection_id}")

Connection estabilshed with the pool pool_b, with ID: 96


In [20]:
# Task 5.2: Create a buffered cursor.

if connection2.is_connected():
    cursor2 = connection2.cursor(buffered=True)
    print('Cursor created.')
else:
    print('The connection is not established')

Cursor created.


In [26]:
# Task 5.3: Combine the data from the Bookings and the Employee tables. Sort the retrieved records in ascending order. Then display the information of the first three guests. 

query_combine = """
select b.*, e.* 
FROM employees e  
INNER JOIN bookings b
ON b.employeeID = e.employeeID
ORDER BY b.bookingID ASC
LIMIT 3;
"""
cursor2.execute(query_combine)

results6 = cursor2.fetchall()
columns6 = cursor2.column_names

df6 = pd.DataFrame(data=results6, columns=columns6)
df6.style.hide(axis='index')

bookingID,tableNo,guestFirstName,guestLastName,bookingSlot,employeeID,employeeID,name,role,address,contact_number,email,annual_salary
1,12,Anna,Iversen,0 days 19:00:00,1,1,Mario Gollini,Manager,"724, Parsley Lane, Old Town, Chicago, IL",351258074,Mario.g@littlelemon.com,70
2,12,Joakim,Iversen,0 days 19:00:00,1,1,Mario Gollini,Manager,"724, Parsley Lane, Old Town, Chicago, IL",351258074,Mario.g@littlelemon.com,70
3,19,Vanessa,McCarthy,0 days 15:00:00,3,3,Giorgos Dioudis,Head Chef,"879 Sage Street, West Loop, Chicago, IL",351970582,Giorgos.d@littlelemon.com,50


In [27]:
# Task 5.4: Return the connection.

if connection2.is_connected():
    connection2.close()
    print('Connection closed.')
else:
    print('The connection do not exist.')

Connection closed.
